In [1]:
import pygame
from pygame import mixer
import random
import math
import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

(6, 0)

In [3]:
screen = pygame.display.set_mode((1000,800))

pygame.display.set_caption("Space Invaders")
icon = pygame.image.load("ufo.png")
pygame.display.set_icon(icon)
background = pygame.image.load("background.png")

In [4]:
mixer.music.load("background.wav")
mixer.music.play(-1)

In [5]:
enemy_dict = {1:"alien0.png",2:"alien1.png",3:"alien2.png",4:"alien3.png",5:"alien4.png",6:"specialalien.png"}

In [6]:
enemyImg = []
enemyX = []
enemyY = []
enemyX_change = []
enemyY_change = []
enemy_appear = False
number_of_enemies = 6

In [7]:
playerImg = pygame.image.load("spaceship.png")
playerX = 440
playerY = 600
playerX_change = 0
playerY_change = 0

In [8]:
bulletImg = pygame.image.load("bullet.png")
bulletY = playerY
bulletY_change = 10
bulletX = 0
bullet_state = 'ready'

In [9]:
for i in range(number_of_enemies):
    enemyImg.append(pygame.image.load(enemy_dict[i+1]))
    if i == 5:
        enemyX_change.append(0)
        enemyY_change.append(1)
        continue
    enemyX.append(random.randint(0,936))
    enemyY.append(random.randint(0,150))
    enemyX_change.append(4)
    enemyY_change.append(40)

enemyX.append(0)
enemyY.append(0)

In [10]:
over_font = pygame.font.Font("Seagram tfb.ttf",64)

In [11]:
def game_over_text():
    over_game = over_font.render("GAME OVER",True,(255,255,255))
    screen.blit(over_game,(250,350))

In [12]:
score_value = 0

font = pygame.font.Font("Seagram tfb.ttf",32)
textX = 10
textY = 10

In [13]:
def show_score(x,y):
    score = font.render("Score : "+ str(score_value),True,(255,255,255))
    screen.blit(score,(x,y))

In [14]:
def fire_bullet(x,y):
    global bullet_state
    bullet_state = 'fire'
    screen.blit(bulletImg,(x+47.5,y))
    

In [15]:
def player(x,y):
    screen.blit(playerImg,(x,y))

In [16]:
def enemy(x,y,i):
        screen.blit(enemyImg[i],(x,y))

In [17]:
def isCollision(enemyX,enemyY,bulletX,bulletY):
    distance = math.sqrt(math.pow((enemyX-bulletX),2) + math.pow((enemyY-bulletY),2))
    if distance<50:
        return True
    else:
        return False

In [18]:
def isGameOver(enemyX,enemyY,playerX,playerY):
    distance = math.sqrt(math.pow((enemyX-playerX),2) + math.pow((enemyY-playerY),2))
    if distance<77.5 or enemyY>=736:
        return True
    else:
        return False

In [19]:
clock = pygame.time.Clock()
destroy_time = 0
passed_time = 0
time = random.randint(0,10)

In [20]:
#Game loop
running = True
while running:
   
    screen.fill((0, 0, 102))
    
    screen.blit(background,(0,0))
    
    for event in pygame.event.get():
        
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
               playerX_change = -5
            if event.key == pygame.K_RIGHT:
               playerX_change = 5
            if event.key == pygame.K_UP:
               playerY_change = -5
            if event.key == pygame.K_DOWN:
               playerY_change = 5 
            if event.key == pygame.K_SPACE:
                if bullet_state == 'ready':
                    bullet_sound = mixer.Sound("laser.wav")
                    bullet_sound.play()
                    fire_bullet(playerX,bulletY)
                    bulletX = playerX
                
        if event.type == pygame.KEYUP:
            playerX_change = 0.0
            playerY_change = 0.0
                    
        if event.type == pygame.QUIT:
            running = False
            break
            
    if enemy_appear == False and passed_time>=time:
        enemy_appear = True
        enemyX[5] = random.randint(0,936)
        enemyY[5] = random.randint(0,150)
        passed_time = 0
        destroy_time = 0
    
    playerX += playerX_change
    playerY += playerY_change
    
    if playerX<=0:
        playerX = 0
    elif playerX>=872:
        playerX = 872
        
    if playerY<=0:
        playerY = 0
    elif playerY>=672:
        playerY = 672
        
    if bulletY<=0:
        bulletY = playerY
        bullet_state = 'ready'
        
    if bullet_state == 'fire':
        fire_bullet(bulletX,bulletY)
        bulletY -= bulletY_change 
        
    for i in range(number_of_enemies):
        if enemyX[i]<=0 and i != 5:
            enemyX_change[i] = -enemyX_change[i] +0.05 
            enemyY[i] += enemyY_change[i]
    
        if enemyX[i]>=936 and i != 5:
            enemyX_change[i] = -enemyX_change[i] - 0.05
            enemyY[i] += enemyY_change[i]
        
        if i != 5:
            enemyX[i] += enemyX_change[i]
        else:
            enemyY[i] += enemyY_change[i]
        
        collision = isCollision(enemyX[i]-16,enemyY[i]-16,bulletX,bulletY)
        game_over = isGameOver(enemyX[i]+32,enemyY[i]+32,playerX+64,playerY+64)
    
        if collision:
            explosion_sound = mixer.Sound("explosion.wav")
            explosion_sound.play()
            bulletY = playerY
            bullet_state = 'ready'
            score_value += 100
            if i == 5:
                enemy_appear = False
                enemyX[5] = 200000
                enemyY[5] = -200000
                destroy_time = pygame.time.get_ticks()
                time = random.randint(0,10)
                continue
            enemyX[i] = random.randint(0,936)
            enemyY[i] = random.randint(0,150)
    
        if game_over:
            for j in range(number_of_enemies):
                enemyY[j] = 20000
            playerY = 25000
            playerX = 25000
            game_over_text()
        
        if enemy_appear == True and i ==5 or i!=5:
            enemy(enemyX[i],enemyY[i],i)
   
    player(playerX,playerY)
    show_score(textX,textY)
    if enemy_appear == False:
        passed_time = (pygame.time.get_ticks() - destroy_time)/1000
    pygame.display.update()
    if not running:
        mixer.music.stop()
        pygame.display.quit()
        
print("GAME OVER!!\n\nScore : ",score_value)
            
    

GAME OVER!!

Score :  14300
